# Cleaning up data for data visualization and gmap's use
* Dropping unneccessary data
* Adding categorizing job post that are not categorized
* Dropping the category unknown 
* Format date created column to datetime


In [27]:
#imports 
import json
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import pprint as pprint

In [28]:
# Raw geo data file to be saved and pulled from
rawGeoData = "../data/rawData/jp_geo_raw_dataframe.csv"

# cleaned data file to be saved
cleanData = "../data/cleanData/jp_clean_data.csv"

# path csv file location
pathRawData = "../data/rawData/"
pathCleanData ="../datacleanData/"

In [29]:
#calling the csv raw data
geo_csv_file = pd.read_csv(rawGeoData,index_col=0)
geo_csv_file.head()

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country
job posting id,,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States


### Counting the columns length to see how much data is dropped during "cleaning"

In [30]:
#Maiking a copy of Geo Data
geo_csv_file_copy = geo_csv_file.copy()
geo_csv_file_copy.count()

job title       983
category        983
company name    983
location        983
date created    983
lat             983
lng             983
latlng          983
geocode_data    983
city            983
state           983
country         983
dtype: int64

In [31]:
#Dropping any rows that are missing 
geo_data = geo_csv_file_copy.dropna()
geo_data.count()

job title       983
category        983
company name    983
location        983
date created    983
lat             983
lng             983
latlng          983
geocode_data    983
city            983
state           983
country         983
dtype: int64

In [32]:
#Calling only Rows that have United states 
dropping_non_us = geo_data[geo_data.country == 'United States']
dropping_non_us.count()

job title       977
category        977
company name    977
location        977
date created    977
lat             977
lng             977
latlng          977
geocode_data    977
city            977
state           977
country         977
dtype: int64

In [33]:
#If there are dupicates Job ID's dorp them and keep the first
clean_data = dropping_non_us.loc[~dropping_non_us.index.duplicated(keep='first')]
clean_data.count()

job title       977
category        977
company name    977
location        977
date created    977
lat             977
lng             977
latlng          977
geocode_data    977
city            977
state           977
country         977
dtype: int64

### Reviewing the how many categories are under the column category

In [35]:
#Check how many different categories you have
checking_cat = clean_data['category'].unique()
list(checking_cat)

['Healthcare & Nursing Jobs',
 'Other/General Jobs',
 'Unknown',
 'Customer Services Jobs',
 'Sales Jobs',
 'Retail Jobs',
 'Logistics & Warehouse Jobs',
 'Accounting & Finance Jobs']

### Create a multiple columns to define job postings that are not defined:
* multiple columns for specific job post and category
* create an empty column called final_c which will be the new Category column
* Append all the new category information to final_c

In [36]:
#Creating columns that have unknown category, define those unknown categories and append to final_c column

#empty column to store the final Category column
clean_data['final_c'] = np.NaN

#Logistics & Warehouse Jobs
warehouse = clean_data[clean_data['job title'].str.contains("Warehous")]
clean_data['warehouse_col'] = warehouse['category'].replace('Unknown', 'Logistics & Warehouse Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['warehouse_col'])

#Customer Services Jobs
shopper = clean_data[clean_data['job title'].str.contains("Shopp")]
clean_data['shopper_col'] = shopper['category'].replace('Unknown', 'Customer Services Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['shopper_col'])

##Healthcare & Nursing Job##

#Physcician
physician = clean_data[clean_data['job title'].str.contains("Physician")]
clean_data['physician_col'] = physician['category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['physician_col'])

#Nurse
nurse = clean_data[clean_data['job title'].str.contains("Nurs")]
clean_data['nurse_col'] = nurse['category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['nurse_col'])

#Teaching Jobs
teach = clean_data[clean_data['job title'].str.contains("Teach")]
clean_data['teacher_col'] = teach['category'].replace('Unknown', 'Teaching Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['teacher_col'])

#Final column Category to append into final_c column
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['category'])

clean_data

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
job posting id,,,,,,,,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320153715,Physician: Interventional Cardiology Opportuni...,Healthcare & Nursing Jobs,CompHealth,"Durant, Bryan County",2019-10-31T23:52:07Z,33.993986,-96.370824,"33.993986,-96.370824","{'address_components': [{'long_name': '507', '...",Durant,Oklahoma,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1338909279,Warehouse Technician - Up to $20.40 HR Salary,Unknown,Amazon,"Cupertino, Santa Clara County",2019-11-14T20:05:27Z,37.322998,-122.032182,"37.322998,-122.032182",{'address_components': [{'long_name': 'Stevens...,Cupertino,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN
1312808260,Physician: Cardiothoracic Surgeon Position in ...,Healthcare & Nursing Jobs,CompHealth,"Wisconsin, US",2019-10-25T12:39:46Z,44.391021,-89.773080,"44.391021,-89.77308000000002","{'address_components': [{'long_name': '312', '...",Wisconsin Rapids,Wisconsin,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN


### Drop the attribute 'Unknown'  from final_c column 

In [37]:
#Drop rows unknown attributes in column final_c

# Get names of indexes for which column Unknown
drop_unknown = clean_data[ clean_data['final_c'] == 'Unknown' ].index
# Delete these row indexes from dataFrame
clean_data.drop(drop_unknown , inplace=True)
clean_data

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
job posting id,,,,,,,,,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338900671,Seasonal Warehousing Worker,Unknown,Amazon,"Oakland, Alameda County",2019-11-14T20:03:27Z,37.772927,-122.214332,"37.772927,-122.21433200000001",{'address_components': [{'long_name': 'Oakland...,Oakland,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN
1320153715,Physician: Interventional Cardiology Opportuni...,Healthcare & Nursing Jobs,CompHealth,"Durant, Bryan County",2019-10-31T23:52:07Z,33.993986,-96.370824,"33.993986,-96.370824","{'address_components': [{'long_name': '507', '...",Durant,Oklahoma,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1338909279,Warehouse Technician - Up to $20.40 HR Salary,Unknown,Amazon,"Cupertino, Santa Clara County",2019-11-14T20:05:27Z,37.322998,-122.032182,"37.322998,-122.032182",{'address_components': [{'long_name': 'Stevens...,Cupertino,California,United States,Logistics & Warehouse Jobs,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN


### Drop the temporary columns that were recently created
* Re-arrang the columns
* Rename the columns

In [42]:
#Drop unwanted columns
dropped_columns = clean_data.drop(columns=['category','warehouse_col', 'location','shopper_col','physician_col','nurse_col','teacher_col','geocode_data'])
dropped_columns.head()

,job title,company name,date created,lat,lng,latlng,city,state,country,final_c
job posting id,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,CompHealth,2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776",Clayton,New York,United States,Healthcare & Nursing Jobs
1312805901,Physician: Urology Need in Western Ohio with L...,CompHealth,2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998",Sunbury,Ohio,United States,Healthcare & Nursing Jobs
1328241931,Physician: Internal Medicine Physician Job in ...,CompHealth,2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",Cheshire,Connecticut,United States,Healthcare & Nursing Jobs
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,CompHealth,2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126",Marion,Illinois,United States,Healthcare & Nursing Jobs
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",CompHealth,2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002",Cromwell,Connecticut,United States,Healthcare & Nursing Jobs


In [43]:
#Re-arrange columns
arrange_col_data = dropped_columns[['job title','date created','final_c','company name','city','state','country','lat','lng','latlng']]
arrange_col_data.head()

,job title,date created,final_c,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,2019-10-25T12:38:41Z,Healthcare & Nursing Jobs,CompHealth,Clayton,New York,United States,44.239491,-76.085776,"44.239490999999994,-76.085776"
1312805901,Physician: Urology Need in Western Ohio with L...,2019-10-25T12:38:57Z,Healthcare & Nursing Jobs,CompHealth,Sunbury,Ohio,United States,40.305232,-82.824785,"40.305232000000004,-82.82478499999998"
1328241931,Physician: Internal Medicine Physician Job in ...,2019-11-06T20:28:13Z,Healthcare & Nursing Jobs,CompHealth,Cheshire,Connecticut,United States,41.508367,-72.910620,"41.508367,-72.91062"
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,2019-11-11T22:10:17Z,Healthcare & Nursing Jobs,CompHealth,Marion,Illinois,United States,37.730605,-88.933126,"37.730605,-88.933126"
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",2019-10-25T12:38:53Z,Healthcare & Nursing Jobs,CompHealth,Cromwell,Connecticut,United States,41.601859,-72.657316,"41.60185900000001,-72.65731600000002"


In [44]:
#Renames column Names
cleansed_data = arrange_col_data.rename(columns={"final_c": "category"})
cleansed_data.head()

,job title,date created,category,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,2019-10-25T12:38:41Z,Healthcare & Nursing Jobs,CompHealth,Clayton,New York,United States,44.239491,-76.085776,"44.239490999999994,-76.085776"
1312805901,Physician: Urology Need in Western Ohio with L...,2019-10-25T12:38:57Z,Healthcare & Nursing Jobs,CompHealth,Sunbury,Ohio,United States,40.305232,-82.824785,"40.305232000000004,-82.82478499999998"
1328241931,Physician: Internal Medicine Physician Job in ...,2019-11-06T20:28:13Z,Healthcare & Nursing Jobs,CompHealth,Cheshire,Connecticut,United States,41.508367,-72.910620,"41.508367,-72.91062"
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,2019-11-11T22:10:17Z,Healthcare & Nursing Jobs,CompHealth,Marion,Illinois,United States,37.730605,-88.933126,"37.730605,-88.933126"
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",2019-10-25T12:38:53Z,Healthcare & Nursing Jobs,CompHealth,Cromwell,Connecticut,United States,41.601859,-72.657316,"41.60185900000001,-72.65731600000002"


### Format date created column to pd.datetime

In [55]:
cleansed_data['date created'] = pd.to_datetime(cleansed_data['date created'], format = '%Y-%m-%dT%H:%M:%SZ')
#raw_data['Mycol'] =  pd.to_datetime(raw_data['Mycol'], format='%d%b%Y:%H:%M:%S.%f')

,job title,date created,category,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805093,Physician: Family Physician Job Opening in Ups...,2019-10-25 12:38:41,Healthcare & Nursing Jobs,CompHealth,Clayton,New York,United States,44.239491,-76.085776,"44.239490999999994,-76.085776"
1312805901,Physician: Urology Need in Western Ohio with L...,2019-10-25 12:38:57,Healthcare & Nursing Jobs,CompHealth,Sunbury,Ohio,United States,40.305232,-82.824785,"40.305232000000004,-82.82478499999998"
1328241931,Physician: Internal Medicine Physician Job in ...,2019-11-06 20:28:13,Healthcare & Nursing Jobs,CompHealth,Cheshire,Connecticut,United States,41.508367,-72.910620,"41.508367,-72.91062"
1334502463,Physician: A Locums Otolaryngologist Is Wanted...,2019-11-11 22:10:17,Healthcare & Nursing Jobs,CompHealth,Marion,Illinois,United States,37.730605,-88.933126,"37.730605,-88.933126"
1312805719,"Physician: Hospitalist needed in Hartford, CT ...",2019-10-25 12:38:53,Healthcare & Nursing Jobs,CompHealth,Cromwell,Connecticut,United States,41.601859,-72.657316,"41.60185900000001,-72.65731600000002"


In [57]:
#Save cleansed data to csv
cleansed_data.to_csv(cleanData)